In [321]:
import sys
import pandas as pd
import datetime as dt
import time
import math
import matplotlib.pyplot as plt
import scipy.stats as stats
from pymongo import MongoClient
from tqdm import tqdm

sys.path.append('..')

from pseudoFrameWork.api import getAnyMOEX

mongoClient = MongoClient('localhost', 27017, username='admin', password='admin')
db = mongoClient['rlab_db']

TODAY = dt.datetime.today()
DATE_FORMAT = '%Y-%m-%d'
DATA_LOAD_LIMIT = 100

In [370]:
# db['moex.futuresList'].delete_many({})

# futuresList = await getAnyMOEX(
#     api_method=r'/iss/statistics/engines/futures/markets/forts/series.json',
#     arguments={},
#     field='series'
# )

# futuresList['loadDate'] = TODAY

# db['moex.futuresList'].insert_many([futuresList.iloc[i].to_dict() for i in range(futuresList.shape[0])])

futuresList = pd.DataFrame(db['moex.futuresList'].find())
futuresList = futuresList[~futuresList['underlying_asset'].isna()]
futuresList.reset_index(drop=True, inplace=True)

# startSecNum = 0

# while startSecNum < len(futuresList):
#     try:
#         for i in tqdm(range(startSecNum, len(futuresList))):
#             numPeriods = math.ceil((TODAY - dt.datetime.strptime(futuresList.iloc[i].start_date, DATE_FORMAT)).days / DATA_LOAD_LIMIT)
#             periods = [(dt.datetime.strptime(futuresList.iloc[i].start_date, DATE_FORMAT) + pd.offsets.Day(DATA_LOAD_LIMIT * period)).strftime(DATE_FORMAT) for period in range(numPeriods)]
#             periods.append(TODAY.strftime(DATE_FORMAT))
        
#             for j in range(len(periods) - 1):
#                 loopData = await getAnyMOEX(
#                     api_method=rf'/iss/history/engines/futures/markets/forts/boards/rfud/securities/{futuresList.iloc[i].secid}.json',
#                     arguments={'from': periods[j], 'till': periods[j+1]},
#                     field='history'
#                 )

#                 loopData['loadDate'] = TODAY

#                 db['moex.futuresMarketData'].insert_many([loopData.iloc[row].to_dict() for row in range(loopData.shape[0])])
            
#             if i >= len(futuresList) - 1:
#                 startSecNum = len(futuresList)
    
#     except Exception:
#         print('Connection was closed... retry...')
#         time.sleep(0.5)

#         db['moex.futuresMarketData'].delete_many({'secid': futuresList.iloc[i].secid})
#         startSecNum = i

futuresMD = pd.DataFrame(db['moex.futuresMarketData'].find())
futuresMD['TRADEDATE'] = pd.to_datetime(futuresMD['TRADEDATE'])

noTradeCount = futuresMD[futuresMD['TRADEDATE'].dt.year == TODAY.year]
noTradeCount = noTradeCount.CLOSE.isnull().groupby(noTradeCount['SECID']).sum().astype(int).reset_index(name='count').sort_values(by='count', ascending=False)

futuresList = futuresList[~futuresList['secid'].isin(noTradeCount[noTradeCount['count'] >= 30]['SECID'])]

volumeTrades = futuresMD[futuresMD['SECID'].isin(futuresList['secid'])]
volumeTrades = volumeTrades.dropna().groupby(volumeTrades['SECID']).agg({'VOLUME': 'mean'}).reset_index()

futuresList = futuresList[futuresList['secid'].isin(volumeTrades[volumeTrades['VOLUME'] >= 100]['SECID'])]

priceSpread = futuresMD[futuresMD['SECID'].isin(futuresList['secid'])].dropna()
priceSpread['normolizeSpread'] = (priceSpread['HIGH'] - priceSpread['LOW']) / priceSpread['CLOSE']
priceSpread = priceSpread.groupby(priceSpread['SECID']).agg({'normolizeSpread': 'mean'}).reset_index()

futuresList = futuresList[futuresList['secid'].isin(priceSpread[priceSpread['normolizeSpread'] < 0.05]['SECID'])]


In [371]:
# plt.figure(figsize=(15, 7))

# secid = 'GAZPF'

# for priceType in ('OPEN', 'HIGH', 'LOW', 'CLOSE'):
#     plt.plot(futuresMD[futuresMD['SECID'] == secid]['TRADEDATE'], futuresMD[futuresMD['SECID'] == secid][priceType], label=priceType)

# plt.legend()
# plt.show()

In [372]:
futuresList

,_id,secid,name,start_date,expiration_date,asset_code,underlying_asset,is_traded,loadDate
0,68d9a4b786a2ceb7162d8655,CNYRUBF,CNYRUBF,2022-04-23,2100-01-01,CNYRUBTOM,CNYRUB_TOM,1,2025-09-28 20:13:16.914
1,68d9a4b786a2ceb7162d8656,EURRUBF,EURRUBF,2022-04-23,2100-01-01,EURRUBTOM,EUR_RUB__TOM,1,2025-09-28 20:13:16.914
2,68d9a4b786a2ceb7162d8657,GAZPF,GAZPF,2024-10-01,2100-01-01,GAZPF,GAZP,1,2025-09-28 20:13:16.914
3,68d9a4b786a2ceb7162d8658,GLDRUBF,GLDRUBF,2023-07-11,2100-01-01,GLDRUBTOM,GLDRUB_TOM,1,2025-09-28 20:13:16.914
4,68d9a4b786a2ceb7162d8659,IMOEXF,IMOEXF,2023-11-14,2100-01-01,IMOEX,IMOEX,1,2025-09-28 20:13:16.914
...,...,...,...,...,...,...,...,...,...
178,68d9a4b786a2ceb7162d87a6,CRZ5,CNY-12.25,2024-06-06,2025-12-18,CNY,CNYRUB_TOM,1,2025-09-28 20:13:16.914
179,68d9a4b786a2ceb7162d87ac,EuZ5,Eu-12.25,2024-06-14,2025-12-18,Eu,EUR_RUB__TOM,1,2025-09-28 20:13:16.914
180,68d9a4b786a2ceb7162d87ae,GLZ5,GL-12.25,2024-12-06,2025-12-18,GL,GLDRUB_TOM,1,2025-09-28 20:13:16.914
182,68d9a4b786a2ceb7162d87b2,IPZ5,IPO-12.25,2025-06-05,2025-12-18,IPO,MIPO,1,2025-09-28 20:13:16.914
